In [16]:
import numpy as np
import torch
import math
import matlab.engine
from scipy.optimize import curve_fit
import data.size_transforms as sizeT

In [2]:
###load data
eng = matlab.engine.start_matlab()

mrs_struct = eng.load("data/gannet_full_load/g7_s01.mat")["mrs_struct"]
diff = np.real(np.array(mrs_struct["spec"]["vox1"]["GABAGlx"]["diff"]))[0]
freq = np.array(mrs_struct["spec"]["freq"])[0]

In [3]:
def gaba_glx_model(freq,x_0,x_1,x_2,x_3,x_4,x_5,x_6,x_7,x_8,x_9,x_10,x_11):
    return x_0*np.exp(x_1*np.square(freq-x_2)) + x_3*np.exp(x_4*np.square(freq-x_5)) + x_6*np.exp(x_7*np.square(freq-x_8)) + x_9*(freq-x_2)+ x_10*np.sin(math.pi*freq/(1.31*4)) + x_11*np.cos(math.pi*freq/(1.31*4))

def gaba_glx_model_weighted(freq,x_0,x_1,x_2,x_3,x_4,x_5,x_6,x_7,x_8,x_9,x_10,x_11):
    w = np.ones(shape=(freq.shape))
    cho_min_ind,cho_max_ind = np.amin(np.argwhere(freq<=3.285)),np.amax(np.argwhere(freq>=3.16))
    w[cho_min_ind:cho_max_ind]=0.001
    return np.sqrt(w)*gaba_glx_model(freq,x_0,x_1,x_2,x_3,x_4,x_5,x_6,x_7,x_8,x_9,x_10,x_11)

In [23]:
def fit_gaba(x,ppm):

    freqbound_min_ind,freqbound_max_ind = np.amin(np.argwhere(ppm<=4.1)),np.amax(np.argwhere(ppm>=2.79))
    gaba_min_ind,gaba_max_ind = np.amin(np.argwhere(ppm<=3.2)),np.amax(np.argwhere(ppm>=2.78))
    glx_min_ind,glx_max_ind = np.amin(np.argwhere(ppm<=4.1)),np.amax(np.argwhere(ppm>=3.4))

    maxin_gaba = x[gaba_min_ind:gaba_max_ind].max()
    maxin_glx = x[glx_min_ind:glx_max_ind].max()

    grad_points = (x[freqbound_max_ind]-x[freqbound_min_ind])/abs(freqbound_max_ind-freqbound_min_ind)
    linear_init = grad_points/abs(ppm[1]-ppm[2])

    gauss_model_init = [maxin_glx,-700,3.71,maxin_glx,-700,3.79,maxin_gaba,-90,3.02,-linear_init,0,0]
    #scaling conditions
    for i in [0,3,6,9]:
        gauss_model_init[i]=gauss_model_init[i]/maxin_glx

    lb = [-4000*maxin_glx, -1000, 3.71-0.02, -4000*maxin_glx, -1000, 3.79-0.02, -4000*maxin_gaba, -200,3.02-0.05,-40*maxin_gaba,-2000*maxin_gaba,-2000*maxin_gaba]
    ub = [4000*maxin_glx, -40, 3.71+0.02, 4000*maxin_glx, -40, 3.79+0.02, 4000*maxin_gaba, -40,3.02+0.05,40*maxin_gaba,1000*maxin_gaba,1000*maxin_gaba]
    for i in [0,3,6,9]:
        lb[i] = lb[i]/maxin_glx
        ub[i] = ub[i]/maxin_glx

    w = np.ones(shape=(x[freqbound_min_ind:freqbound_max_ind].shape))
    cho_min_ind,cho_max_ind = np.amin(np.argwhere(ppm<=3.285)),np.amax(np.argwhere(ppm>=3.16))
    w[cho_min_ind:cho_max_ind]=0.001


    print(gauss_model_init)
    gauss_model_init,_ = curve_fit(gaba_glx_model,ppm[freqbound_min_ind:freqbound_max_ind],x[freqbound_min_ind:freqbound_max_ind]/maxin_glx,gauss_model_init,bounds=(lb,ub))
    print(gauss_model_init)
    print("------")
    gauss_model_param,_ = curve_fit(gaba_glx_model_weighted,ppm[freqbound_min_ind:freqbound_max_ind],np.sqrt(w)*x[freqbound_min_ind:freqbound_max_ind]/maxin_glx,gauss_model_init,bounds=(lb,ub))
    #print(gauss_model_init)
    print(gauss_model_param)

    #rescale
    for i in [0,3,6,9]:
        gauss_model_param[i] = gauss_model_param[i]*maxin_glx

    area = gauss_model_param[6] / np.sqrt(-gauss_model_param[7]) * np.sqrt(math.pi)
    height = gauss_model_param[6]

    print(area)
    print(height)

In [25]:
diff_resize = sizeT.spec_resampling_1d(torch.from_numpy(diff),torch.from_numpy(freq),sizeT.ref_freq_short)

y2 = fit_gaba(diff_resize.numpy(),sizeT.ref_freq_short.numpy())

[1.0, -700, 3.71, 1.0, -700, 3.79, 0.6574454520292433, -90, 3.02, -0.04353306245328286, 0, 0]
[ 8.31039644e-01 -6.72825544e+02  3.70897548e+00  8.99239114e-01
 -3.72589881e+02  3.78102024e+00  6.72060896e-01 -6.65069899e+01
  2.99378684e+00 -2.88872427e-01 -3.37269425e-01 -4.37109175e-01]
------
[   0.84260165 -651.98618604    3.70916812    0.89763202 -379.56189898
    3.78147831    0.69458162  -68.85873819    2.99373951   -1.23843745
   -1.34780756   -1.74907086]
0.00040991028838970755
0.001919081472406676


In [24]:
y = fit_gaba(diff,freq)

[1.0, -700, 3.71, 1.0, -700, 3.79, 0.6585623938862255, -90, 3.02, -0.04542326084698159, 0, 0]
[ 8.31544717e-01 -6.72494888e+02  3.70899613e+00  8.98964937e-01
 -3.73282845e+02  3.78104114e+00  6.68912143e-01 -6.74740614e+01
  2.99419269e+00 -5.04406620e-01 -5.56927281e-01 -7.24544446e-01]
------
[   0.84404964 -650.18235041    3.7092076     0.89712261 -381.04964442
    3.78153424    0.69324328  -71.04155919    2.99405902   -1.60318228
   -1.72301448   -2.23918161]
0.0004027955529997926
0.0019154286235860435
